In [1]:
!pip install datasets==3.6.0

## Training a new tokenizer

In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("code_search_net", "python")

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


data/python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [3]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [4]:
print(raw_datasets["train"][0]["whole_func_string"])

async def ensure_open(self) -> None:
        """
        Check that the WebSocket connection is open.

        Raise :exc:`~websockets.exceptions.ConnectionClosed` if it isn't.

        """
        # Handle cases from most common to least common for performance.
        if self.state is State.OPEN:
            # If self.transfer_data_task exited without a closing handshake,
            # self.close_connection_task may be closing it, going straight
            # from OPEN to CLOSED.
            if self.transfer_data_task.done():
                await asyncio.shield(self.close_connection_task)
                raise ConnectionClosed(
                    self.close_code, self.close_reason
                ) from self.transfer_data_exc
            else:
                return

        if self.state is State.CLOSED:
            raise ConnectionClosed(
                self.close_code, self.close_reason
            ) from self.transfer_data_exc

        if self.state is State.CLOSING:
         

### **Using a Python generator, we can avoid Python loading anything into memory until it’s actually necessary. To create such a generator, you just to need to replace the brackets with parentheses:**

In [5]:
raw_datasets["train"].num_rows/1000

412.178

In [6]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

In [7]:
# traing_corpus = (raw_datasets["train"][x:x+1000]["whole_func_string"]  for x in range(0,len(raw_datasets["train"]),1000))
print(next(training_corpus)[0])

async def ensure_open(self) -> None:
        """
        Check that the WebSocket connection is open.

        Raise :exc:`~websockets.exceptions.ConnectionClosed` if it isn't.

        """
        # Handle cases from most common to least common for performance.
        if self.state is State.OPEN:
            # If self.transfer_data_task exited without a closing handshake,
            # self.close_connection_task may be closing it, going straight
            # from OPEN to CLOSED.
            if self.transfer_data_task.done():
                await asyncio.shield(self.close_connection_task)
                raise ConnectionClosed(
                    self.close_code, self.close_reason
                ) from self.transfer_data_exc
            else:
                return

        if self.state is State.CLOSED:
            raise ConnectionClosed(
                self.close_code, self.close_reason
            ) from self.transfer_data_exc

        if self.state is State.CLOSING:
         

In [8]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

old_tokens = old_tokenizer.tokenize(example)
print(old_tokens)

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


In [10]:
old_tokenizer.model_max_length # this is the context size of the gpt2

1024

In [11]:
old_tokenizer.vocab_size # vocabulary size of gpt2 model tokenizer

50257

In [12]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [13]:
tokens = tokenizer.tokenize(example)
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [14]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
print(tokenizer.tokenize(example))

['class', 'ĠLinear', 'Layer', '():', 'ĊĠĠĠ', 'Ġdef', 'Ġ__', 'init', '__(', 'self', ',', 'Ġinput', '_', 'size', ',', 'Ġoutput', '_', 'size', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'weight', 'Ġ=', 'Ġtorch', '.', 'randn', '(', 'input', '_', 'size', ',', 'Ġoutput', '_', 'size', ')', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'bias', 'Ġ=', 'Ġtorch', '.', 'zeros', '(', 'output', '_', 'size', ')', 'ĊĊĠĠĠ', 'Ġdef', 'Ġ__', 'call', '__(', 'self', ',', 'Ġx', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġreturn', 'Ġx', 'Ġ@', 'Ġself', '.', 'weights', 'Ġ+', 'Ġself', '.', 'bias', 'ĊĠĠĠĠ']


In [15]:
tokenizer.save_pretrained("python-code-search-tokenizer")

('python-code-search-tokenizer/tokenizer_config.json',
 'python-code-search-tokenizer/special_tokens_map.json',
 'python-code-search-tokenizer/vocab.json',
 'python-code-search-tokenizer/merges.txt',
 'python-code-search-tokenizer/added_tokens.json',
 'python-code-search-tokenizer/tokenizer.json')

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
# tokenizer.push_to_hub("python-code-search-tokenizer")

Training GPT

In [18]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})

In [19]:
new_dataset= raw_datasets["train"].shuffle(seed=42).select(range(2))
new_dataset = new_dataset.map(lambda x: {"text":x["whole_func_string"]},remove_columns=raw_datasets["train"].features)
new_dataset

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 2
})

In [20]:
# def tokenize_and_split(examples):
#     return tokenizer(
#         examples["text"],
#         truncation=True,
#         max_length=128,
#         return_overflowing_tokens=True,
#         return_length=True,
#     )

In [21]:
# a=tokenize_and_split(new_dataset[1])
# a

In [22]:
# print(new_dataset[1]["text"])

In [23]:
# print(tokenizer.decode(a["input_ids"][0]))

In [24]:
# tok = new_dataset.map(
#     tokenize_and_split, batched=True,remove_columns=new_dataset.column_names
# )

In [25]:
# for x in tok:
#   print(x["length"])

In [26]:
context_length=128
def tokenize(element):
    outputs = tokenizer(
        element["func_code_string"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

Map:   0%|          | 0/22176 [00:00<?, ? examples/s]

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 603022
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 32912
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 37680
    })
})

In [27]:
print(tokenizer.decode(tokenized_datasets["train"][0]["input_ids"]))

async def ensure_open(self) -> None:
        """
        Check that the WebSocket connection is open.

        Raise :exc:`~websockets.exceptions.ConnectionClosed` if it isn't.

        """
        # Handle cases from most common to least common for performance.
        if self.state is State.OPEN:
            # If self.transfer_data_task exited without a closing handshake,
            # self.close_connection_task may be closing it, going straight
            # from OPEN to CLOSED.
            if self.transfer_data_task.done():
                await asyncio.shield(self.close_connection_task


In [28]:
tokenizer.bos_token_id


0

In [29]:
from transformers import AutoTokenizer, AutoConfig, GPT2LMHeadModel

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [30]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 125.8M parameters


In [31]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [32]:
out = data_collator([tokenized_datasets["train"][i] for i in range(2)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([2, 128])
attention_mask shape: torch.Size([2, 128])
labels shape: torch.Size([2, 128])


In [33]:
from huggingface_hub import notebook_login
notebook_login()

In [34]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="python_codeparrot",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

/tmp/ipython-input-3505802723.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [35]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azheraly009 (azheraly009-nust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


TrainOutput(global_step=2356, training_loss=3.6068979467317486, metrics={'train_runtime': 6894.7564, 'train_samples_per_second': 87.461, 'train_steps_per_second': 0.342, 'total_flos': 3.9391210930176e+16, 'train_loss': 3.6068979467317486, 'epoch': 1.0})

In [36]:
trainer.push_to_hub()

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...python_codeparrot/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ...vents.1756528666.39eacd098816.955.0: 100%|##########| 5.54kB / 5.54kB            

  ...python_codeparrot/model.safetensors:   7%|6         | 33.5MB /  503MB            

CommitInfo(commit_url='https://huggingface.co/azherali/python_codeparrot/commit/4ff2caec0271ba6fecada5efa37b456fa55bd376', commit_message='End of training', commit_description='', oid='4ff2caec0271ba6fecada5efa37b456fa55bd376', pr_url=None, repo_url=RepoUrl('https://huggingface.co/azherali/python_codeparrot', endpoint='https://huggingface.co', repo_type='model', repo_id='azherali/python_codeparrot'), pr_revision=None, pr_num=None)

In [37]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model="azherali/python_codeparrot", device=device
)

config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/503M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Device set to use cuda


In [46]:
# we have have to train this model for more epoch this time we have train for only one epoch which shows not good results